In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import codecs
import time
import re
import os
from IPython.display import display, HTML
from difflib import SequenceMatcher
import sys

In [2]:
# sys.path.append('/sanskrit_transcoder')
# import transcoder


In [3]:
import importlib.util
spec = importlib.util.spec_from_file_location("module.name","sanskrit_transcoder/transcoder.py")
transcoder = importlib.util.module_from_spec(spec)
spec.loader.exec_module(transcoder)
transcoder.transcoder_set_dir('sanskrit_transcoder/data/transcoder');

In [6]:
s = pd.read_csv("all_sandhi.txt",encoding = 'utf-8' ,sep = ',')
s

,id,c1,c2,c3
0,0,a,a,A
1,1,a,A,A
2,2,a,i,e
3,3,a,I,e
4,4,a,u,o
5,5,a,U,o
6,6,a,q,ar
7,7,a,q,a q
8,8,a,Q,ar
9,9,a,L,al


In [263]:
## sandhi implemented here .
names = ["workdata/lines_submit/correct_lines/sorteddata/","workdata/lines_submit/wrong_lines/sorteddata/"]

# fnames = os.listdir("workdata/lines_submit/correct_lines/sorteddata/")
for name in names :
    fnames = os.listdir(name)
    for fname in fnames:
        if fname == ".ipynb_checkpoints":
            continue
#         print(fname+"/dataframe.txt")
        with open(name+fname+"/input_line.txt",encoding = 'utf-8' ,mode ='r') as fr :
            lines = fr.readlines()
        for line in lines :
            if(line.startswith("converted line : ")) :
                line1 = line[17:]
                break
        chunkwords = line1.split("\xa0")
#         print(chunkwords)
        df = pd.read_csv(name+fname+"/dataframe.txt",encoding = 'utf-8' ,sep = ',')
        cf = pd.read_csv(name+fname+"/conflicts.csv",encoding = 'utf-8' ,sep = ',')
        df['word_slp1'] = df['word']
        for i in df.index:
            df.loc[i,'word_slp1']   =  transcoder.transcoder_processString(df.loc[i,'word'],'roman','slp1')  ##transcoder
        df['sandhi_indexs'] = ""
        df['sandhi_words'] = ""
        for i in df.index : 
            word1 = df.loc[i,'word_slp1']
            chunkno = df.loc[i,'chunk_no']
            position1 =  df.loc[i,'position']
            level = df.loc[i,'level']
            color1 = df.loc[i,'color_class']
            clspan1 = df.loc[i,'colspan']
            chunkword = transcoder.transcoder_processString(chunkwords[chunkno-1],'roman','slp1')   ## transcoder
            df1 = df.loc[(df['chunk_no'] == chunkno) & (df['level']!=level)]
            for j in df1.index :
                if (cf.loc[i,str(j)] != 1) and (cf.loc[i,str(j)] != 0):
                    cf.loc[i,str(j)] = 2
                position2 = df1.loc[j,'position']
                color2 = df.loc[i,'color_class']
                word2 = df1.loc[j,'word_slp1']
                word3 = df1.loc[j,'word']
                if(position2 == position1 + clspan1-1) and (clspan1!=1):
                    if((color1=='yellow_back') and (color2=='carmin_back'))  :    ## check color
                        continue
                    if((color2=='yellow_back') and (color1=='carmin_back'))  :    ## check color
                        continue
                    if(len(s.loc[(s['c1']==word1[-1:] )& (s['c2']==word2[:1])])):
                        
                        x=s.loc[(s['c1']==word1[-1:] )& (s['c2']==word2[:1])]
                        for c3 in x['c3']:
                            mergeword = word1[:-1] + c3 + word2[1:]
#                             print(mergeword+"---"+chunkword)
                            if(re.findall(mergeword,chunkword)):               ## checking  mergedword(spl1) in chunkword
                                df.loc[i,'sandhi_indexs'] += str(j)+", "
                                df.loc[i,'sandhi_words'] += word3+"("+word2+"), "
                                cf.loc[i,str(j)] = 1                             ## merge pssible so change conflict
                                cf.loc[j,str(i)] = 1   
#                                 print(cf.loc[i,str(j)])
                                break
                    elif(word1[-1:]=='H'):
                        if(len(s.loc[(s['c1']==word1[-2:] )& (s['c2']==word2[:1])])):
                        
                            x=s.loc[(s['c1']==word1[-2:] )& (s['c2']==word2[:1])]
                            for c3 in x['c3']:
                                mergeword = word1[:-1] + c3 + word2[1:]
                                if(re.findall(mergeword,chunkword)):          ## checking  mergedword(spl1) in chunkword   
                                    df.loc[i,'sandhi_indexs'] += str(j)+", "
                                    df.loc[i,'sandhi_words'] += word3+"("+word2+"), "
                                    cf.loc[i,str(j)] = 1                        ## merge pssible so change conflict
                                    cf.loc[j,str(i)] = 1 
                                    break
                elif(position2 > position1 + clspan1-1) :
                    cf.loc[i,str(j)] = 0
                    cf.loc[j,str(i)] = 0
        
        df.to_csv(name+fname+"/dataframe_withsandhi.txt" ,encoding = 'utf-8' , sep=',',index = False,mode = 'w' )
        
       ## for conflict finding 0 = no conflict,1=possible merge,2=conflict 
        for i in df.index :
            chunkno = df.loc[i,'chunk_no']
            position1 =  df.loc[i,'position']
            level = df.loc[i,'level']
            color1 = df.loc[i,'color_class']
            clspan1 = df.loc[i,'colspan']
            df1 = df.loc[(df['chunk_no'] != chunkno)]
            for j in df1.index :
                cf.loc[i,str(j)] = 0
            df1 = df.loc[(df['level']==level) & (df['chunk_no'] == chunkno)]
            for j in df1.index :
                cf.loc[i,str(j)] = 0

        cf.to_csv(name+fname+"/conflicts.csv" ,encoding = 'utf-8' , sep=',',index = False,mode = 'w' )               
                        
#         break
#     break
# print(name+fname+"/dataframe_withsandhi.txt")

In [285]:
# getting undefined words,without conflict sentences for partial,complete analasys

names = ["workdata/lines_submit/correct_lines/sorteddata/","workdata/lines_submit/wrong_lines/sorteddata/"]

os.makedirs('workdata/lines_submit/analasys/', exist_ok=True)

pbwords = []

clinefolder = []
wlinefolder = []
                  
for name in names : 
    fnames = os.listdir(name)
    for fname in fnames:
        if fname == ".ipynb_checkpoints":
            continue
        if(name == names[1]):
            
            with open(name+fname+"/problem_words.txt",encoding = 'utf-8' ,mode ='r') as fr :
                c =1
                for line in fr.readlines() :
                    if c==1 :
                        c=0
                        continue
#                     if not line[:-1] in pbwords :
                    pbwords.append(line[:-1])
        
        cf = pd.read_csv(name+fname+"/conflicts.csv",encoding = 'utf-8' ,sep = ',')
        bl = 0
        for i in range(0,len(cf)):
            for j in range(0,len(cf)):
                if not cf.loc[i,str(j)]==0:
                    bl=1
        if(bl == 0):
            if(name==names[0]):
                clinefolder.append(name+fname)
            else :
                wlinefolder.append(name+fname)
    if(name==names[0]):
        with open("workdata/lines_submit/analasys/correctlines_noconflics_folders.txt",encoding = 'utf-8' ,mode ='w') as fw :
            fw.writelines("\n".join(clinefolder))
    else :
        with open("workdata/lines_submit/analasys/wronglines_noconflics_folders.txt",encoding = 'utf-8' ,mode ='w') as fw :
            fw.writelines("\n".join(wlinefolder))
os.makedirs('workdata/lines_submit/analasys/', exist_ok=True)
with open("workdata/lines_submit/analasys/undifined_words.txt",encoding = 'utf-8' ,mode ='w') as fw :
    fw.writelines("\n".join(pbwords))
                         

In [282]:
with open("workdata/lines_submit/analasys/undifined_words.txt",encoding = 'utf-8' ,mode ='r') as fr :
    pbwords = fr.readlines()
pf = pd.DataFrame(index=range(0,1))
for pbw in pbwords:
    pf[pbw[:-1]] = 0
for pbw in pbwords:
    pf[pbw[:-1]] += 1
pf.to_csv("workdata/lines_submit/analasys/undifined_wordsfreaquency.csv",encoding = 'utf-8' , sep=',',index = False,mode = 'w' ) 

In [286]:
len(pbwords
   )

1137

In [187]:
for name in names :
    fnames = os.listdir(name)
    for fname in fnames:
        if fname == ".ipynb_checkpoints":
            continue
        df1 = pd.read_csv(name+fname+"/dataframe.txt",encoding = 'utf-8' ,sep = ',')
        df2 = df1 = pd.read_csv(name+fname+"/dataframe_withsandhi.txt",encoding = 'utf-8' ,sep = ',')
        df1.to_csv(name+fname+"/dataframe.csv" ,encoding = 'utf-8' , sep=',',index = False,mode = 'w' )
        df2.to_csv(name+fname+"/dataframe_withsandhi.csv" ,encoding = 'utf-8' , sep=',',index = False,mode = 'w' )

In [7]:
# ## sandhi implemented here for modifies wronglines .
# names = ["workdata/lines_submit/modified_wrong_lines/correct_lines/sorteddata/","workdata/lines_submit/modified_wrong_lines/wrong_lines/sorteddata/"]

# # fnames = os.listdir("workdata/lines_submit/correct_lines/sorteddata/")
# for name in names :
#     fnames = os.listdir(name)
#     for fname in fnames:
#         if fname == ".ipynb_checkpoints":
#             continue
# #         print(fname+"/dataframe.txt")
#         with open(name+fname+"/input_line.txt",encoding = 'utf-8' ,mode ='r') as fr :
#             lines = fr.readlines()
#         for line in lines :
#             if(line.startswith("converted line : ")) :
#                 line1 = line[17:]
#                 break
#         chunkwords = line1.split("\xa0")
# #         print(chunkwords)
#         df = pd.read_csv(name+fname+"/dataframe.txt",encoding = 'utf-8' ,sep = ',')
#         cf = pd.read_csv(name+fname+"/conflicts.csv",encoding = 'utf-8' ,sep = ',')
#         df['word_slp1'] = df['word']
#         for i in df.index:
#             df.loc[i,'word_slp1']   =  transcoder.transcoder_processString(df.loc[i,'word'],'roman','slp1')  ##transcoder
#         df['sandhi_indexs'] = ""
#         df['sandhi_words'] = ""
#         for i in df.index : 
#             word1 = df.loc[i,'word_slp1']
#             chunkno = df.loc[i,'chunk_no']
#             position1 =  df.loc[i,'position']
#             level = df.loc[i,'level']
#             color1 = df.loc[i,'color_class']
#             clspan1 = df.loc[i,'colspan']
#             chunkword = transcoder.transcoder_processString(chunkwords[chunkno-1],'roman','slp1')   ## transcoder
#             df1 = df.loc[(df['chunk_no'] == chunkno) & (df['level']!=level)]
#             for j in df1.index :
#                 if (cf.loc[i,str(j)] != 1) and (cf.loc[i,str(j)] != 0):
#                     cf.loc[i,str(j)] = 2
#                 position2 = df1.loc[j,'position']
#                 color2 = df.loc[i,'color_class']
#                 word2 = df1.loc[j,'word_slp1']
#                 word3 = df1.loc[j,'word']
#                 if(position2 == position1 + clspan1-1) and (clspan1!=1):
#                     if((color1=='yellow_back') and (color2=='carmin_back'))  :    ## check color
#                         continue
#                     if((color2=='yellow_back') and (color1=='carmin_back'))  :    ## check color
#                         continue
#                     if(len(s.loc[(s['c1']==word1[-1:] )& (s['c2']==word2[:1])])):
                        
#                         x=s.loc[(s['c1']==word1[-1:] )& (s['c2']==word2[:1])]
#                         for c3 in x['c3']:
#                             mergeword = word1[:-1] + c3 + word2[1:]
# #                             print(mergeword+"---"+chunkword)
#                             if(re.findall(mergeword,chunkword)):               ## checking  mergedword(spl1) in chunkword
#                                 df.loc[i,'sandhi_indexs'] += str(j)+", "
#                                 df.loc[i,'sandhi_words'] += word3+"("+word2+"), "
#                                 cf.loc[i,str(j)] = 1                             ## merge pssible so change conflict
#                                 cf.loc[j,str(i)] = 1   
# #                                 print(cf.loc[i,str(j)])
#                                 break
#                     elif(word1[-1:]=='H'):
#                         if(len(s.loc[(s['c1']==word1[-2:] )& (s['c2']==word2[:1])])):
                        
#                             x=s.loc[(s['c1']==word1[-2:] )& (s['c2']==word2[:1])]
#                             for c3 in x['c3']:
#                                 mergeword = word1[:-1] + c3 + word2[1:]
#                                 if(re.findall(mergeword,chunkword)):          ## checking  mergedword(spl1) in chunkword   
#                                     df.loc[i,'sandhi_indexs'] += str(j)+", "
#                                     df.loc[i,'sandhi_words'] += word3+"("+word2+"), "
#                                     cf.loc[i,str(j)] = 1                        ## merge pssible so change conflict
#                                     cf.loc[j,str(i)] = 1 
#                                     break
#                 elif(position2 > position1 + clspan1-1) :
#                     cf.loc[i,str(j)] = 0
#                     cf.loc[j,str(i)] = 0
        
#         df.to_csv(name+fname+"/dataframe_withsandhi.txt" ,encoding = 'utf-8' , sep=',',index = False,mode = 'w' )
        
#        ## for conflict finding 0 = no conflict,1=possible merge,2=conflict 
#         for i in df.index :
#             chunkno = df.loc[i,'chunk_no']
#             position1 =  df.loc[i,'position']
#             level = df.loc[i,'level']
#             color1 = df.loc[i,'color_class']
#             clspan1 = df.loc[i,'colspan']
#             df1 = df.loc[(df['chunk_no'] != chunkno)]
#             for j in df1.index :
#                 cf.loc[i,str(j)] = 0
#             df1 = df.loc[(df['level']==level) & (df['chunk_no'] == chunkno)]
#             for j in df1.index :
#                 cf.loc[i,str(j)] = 0

#         cf.to_csv(name+fname+"/conflicts.csv" ,encoding = 'utf-8' , sep=',',index = False,mode = 'w' )               
                        
# #         break
# #     break
# # print(name+fname+"/dataframe_withsandhi.txt")